# Upload pre-annotated images

In [ ]:
from datalabeling.annotator import Annotator
# import pprint
# import os
# from dotenv import load_dotenv 
# from label_studio_ml.utils import get_local_path
# from label_studio_sdk import Client
# from PIL import Image
# from time import time
# from tqdm import tqdm

In [2]:
handler = Annotator(path_to_weights=...,
                    mlflow_model_version='5',
                    dotenv_path='../.env')

Device: cpu


In [3]:
handler.upload_predictions(project_id=3,top_n=2)

Uploading predictions:   0%|          | 0/2 [00:00<?, ?it/s]

Performing prediction on 42 slices.


Uploading predictions:  50%|█████     | 1/2 [00:17<00:17, 17.61s/it]

Performing prediction on 42 slices.


Uploading predictions: 100%|██████████| 2/2 [00:34<00:00, 17.32s/it]


# Loading Savmap dataset

In [59]:
import geopandas as gdp
from PIL import Image
from tqdm import tqdm
import os
from pathlib import Path
import numpy as np
import json
import pandas as pd

In [60]:
#  load data
root = "/Users/sfadel/Desktop/savmap"
polygons = gdp.read_file(os.path.join(root,'savmap_annotations_2014.geojson'))

path_to_images = root #os.path.join(root,'images')
path_to_labels = os.path.join(root,'labels')

In [62]:
# get bounding boxs
for cat in ['x_min', 'y_min', 'x_max', 'y_max','width','height','x','y']:
    polygons[cat] = None
for i in range(len(polygons)):
    x_min, y_min, x_max, y_max = polygons['geometry'].iloc[i].bounds
    image_path = os.path.join(path_to_images,f"{polygons['IMAGEUUID'].iloc[i]}.JPG")
    try:
        width, height = Image.open(image_path).size
    except:
        continue
    polygons['x_min'].iat[i] = int(x_min)
    polygons['x_max'].iat[i] = int(x_max)
    polygons['y_min'].iat[i] = int(y_min)
    polygons['y_max'].iat[i] = int(y_max)
    polygons['x'].iat[i] = round(0.5*(x_max+x_min))
    polygons['y'].iat[i] = round(0.5*(y_max+y_min))
    polygons['width'] = width
    polygons['height'] = height

polygons['bbox_w'] = polygons['x_max'] - polygons['x_min']
polygons['bbox_h'] = polygons['y_max'] - polygons['y_min']
polygons['class'] = 0

In [64]:
polygons.head(2)

,IMAGEUUID,TAGUUID,geometry,x_min,y_min,x_max,y_max,width,height,x,y,bbox_w,bbox_h,class
0,f77f4af5a1344b9086b307d2b4ba61ff,a9b3a2325dbe4a208bc3ae37eeb8e1e1,"POLYGON ((1197 568, 1186 568, 1179 582, 1190 5...",None,None,None,None,4608,3456,None,None,NaN,NaN,0
1,33c79ba79aca4b06ae30a109e5cd868f,4999d35b3b8b407f8cbf48d95e689899,"POLYGON ((2689 778, 2674 747, 2697 741, 2711 7...",None,None,None,None,4608,3456,None,None,NaN,NaN,0


In [79]:
# Select labels
data = polygons.loc[:,['x_min','x_max','y_min','y_max','IMAGEUUID','width','height','class']].copy()
data.rename(columns={'IMAGEUUID':'filename',
                     'x_max':'xmax',
                     'x_min':'xmin',
                     'y_min':'ymin',
                     'y_max':'ymax'},inplace=True)
data['filename'] = data['filename'].apply(lambda x: f"{x}.JPG")
data = data.dropna()
data = data[['filename','class','width', 'height','xmin','ymin','xmax','ymax']]
data.head(1)


,filename,class,width,height,xmin,ymin,xmax,ymax
52,2acac7c355f4454a96a6b60b813a1a1a.JPG,0,4608,3456,3298,2028,3345,2066


In [106]:
# Convert CSV to COCO
save_json_path = ... #'cocoformat.json'

images = []
categories = []
annotations = []

category = {}
category["supercategory"] = 'None'
category["id"] = 0
category["category_name"] = 'wildlife'
categories.append(category)

data['fileid'] = data['filename'].astype('category').cat.codes
data['categoryid']= pd.Categorical(data['class'],ordered= True).codes
# data['categoryid'] = data['categoryid']+1
data['annid'] = data.index

def image(row):
    image = {}
    image["height"] = row.height
    image["width"] = row.width
    image["id"] = row.fileid
    image["file_name"] = row.filename
    return image

# def category(row):
#     category = {}
#     category["supercategory"] = 'None'
#     category["id"] = row.categoryid
#     category["category_name"] = row[2]
#     return category

def annotation(row):
    annotation = {}
    area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
    annotation["segmentation"] = []
    annotation["iscrowd"] = 0
    annotation["area"] = area
    annotation["image_id"] = row.fileid
    annotation["score"] = 1.0

    annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

    annotation["category_id"] = row.categoryid
    annotation["id"] = row.annid
    return annotation

for row in data.itertuples():
    annotations.append(annotation(row))

imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
for row in imagedf.itertuples():
    images.append(image(row))

# catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
# for row in catdf.itertuples():
#     categories.append(category(row))

data_coco = {}
data_coco["images"] = images
data_coco["categories"] = categories
data_coco["annotations"] = annotations
# json.dump(data_coco, open(save_json_path, "w"), indent=4)

In [118]:
groundtruth = {annot['file_name']:[] for annot in data_coco['images']}
for annot,image_data in zip(data_coco['annotations'],data_coco['images']):
    annot.update(image_data)
    # pprint.pp(annot)
    annot['category_name'] = category['category_name']
    groundtruth[annot['file_name']].append(annot)

In [ ]:
groundtruth

In [130]:
# provide correct alias, "pt", "onnx"
handler = Annotator(mlflow_model_alias='pt',
                    mlflow_model_version="groundtruth")
directory_preds = handler.build_upload_json(path_img_dir='/Users/sfadel/Desktop/savmap',
                                            root='/Users/sfadel',
                                            bulk_predictions=groundtruth)

In [132]:
# directory_preds

In [134]:
# Uncomment to save data
# with open('/Users/sfadel/Desktop/savmap/savmap_annotations_2014_labelstudio.json','w') as file:
#     json.dump(directory_preds,file,indent=2)

# Image tiling for annotation

#### Meanings of arguments
- ```-ratioheight``` : proportion of tile  w.r.t height of image. Example 0.5 means dividing the image in two bands w.r.t height.
- ```-ratiowidth``` : proportion of tile w.r.t to width of image. Example 1.0 means the width of the tile is the same as the image.
- ```-overlapfactor``` : percentage of overlap. It should be less than 1.
- ```-rmheight``` : percentage of height to remove or crop at bottom and top
- ```-rmwidth``` : percentage of width to remove or crop on each side of the image
- ```-pattern``` : "**/*.JPG" will get all .JPG images in directory and subdirectories. On windows it will get both .JPG and .jpg. On unix it will only get .JPG images


In [ ]:
# New script for tiling data
# images_to_tile = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images"
# destination_directory = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images-tiled"
!python ../../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Dry season\Leopard rock\Camp 22 + 37-40\Rep 2" 0 0 0 -overlapfactor 0.1  -ratiowidth 0.33334 -ratioheight 0.5 -rmheight 0.21 -rmwidth 0.08 -dest "D:\PhD\Data per camp\Dry season\Leopard rock\Camp 22 + 37-40\Rep 2 - tiled" -pattern "**/*.JPG"

# Pre-annotating data for labelstudio

In [ ]:
from preprocessing import Annotator
import os

In [ ]:
# Example
# provide correct alias, "pt", "onnx"
handler = Annotator(mlflow_model_alias='cycle1')
path_img_dir=r"D:\PhD\Data per camp\Dry season\Leopard rock\Camp 22 37-40\Rep 2 - tiled"
root="D:\\"
save_json_path = os.path.join(Path(path_img_dir).parent,
                              f"{Path(path_img_dir).name}_preannotation_label-studio.json")
directory_preds = handler.build_upload_json(path_img_dir=path_img_dir,
                                            root=root,
                                            save_json_path=save_json_path,
                                            pattern="**/*.JPG")

In [ ]:
# from preprocessing import Annotator
# from tqdm import tqdm
# import os
# from dotenv import load_dotenv 
# from label_studio_ml.utils import get_local_path
# from label_studio_sdk import Client
# from PIL import Image
# from tqdm import tqdm


# def upload_predictions(project_id:int,
#                        annotator:Annotator,
#                        dotenv_path:str):

#     # Load environment variables
#     load_dotenv(dotenv_path=dotenv_path)

#     # Connect to the Label Studio API and check the connection
#     LABEL_STUDIO_URL = os.getenv('LABEL_STUDIO_URL')
#     API_KEY = os.getenv("LABELSTUDIO-API-KEY")      
#     ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

#     # Select project
#     project = ls.get_project(id=project_id)

#     # Upload predictions for each task
#     tasks = project.get_tasks()
#     for task in tqdm(tasks,desc="Uploading predictions"):
#         task_id = task['id']
#         img_url = task['data']['image']
#         img_path = get_local_path(img_url)
#         img = Image.open(img_path)
#         prediction = annotator.predict(img)
#         img_width, img_height = img.size
#         formatted_pred = [annotator.format_prediction(pred,
#                                                     img_height=img_height,
#                                                     img_width=img_width) for pred in prediction]
#         conf_scores = [pred['score'] for pred in prediction]
#         max_score = 0.0
#         if len(conf_scores)>0:
#             max_score = max(conf_scores)
#         project.create_prediction(task_id=task_id,
#                             score=max_score,
#                             result=formatted_pred,
#                             model_version=annotator.modelversion)

In [ ]:
# upload_predictions(project_id=37,
#                    annotator=Annotator(mlflow_model_alias='cycle1'),
#                    dotenv_path="../.env")

In [ ]:
from sahi.models.yolov8 import Yolov8DetectionModel
from ultralytics import YOLO
from sahi.predict import get_sliced_prediction
import torch
from PIL import Image
from time import time
from pathlib import Path
import mlflow
import json
import pprint
from tqdm import tqdm
import os
from dotenv import load_dotenv 
from label_studio_ml.utils import get_local_path
from label_studio_sdk import Client
# from preprocessing import Annotator
from copy import deepcopy

class Annotator(object):

    def __init__(self,
                dotenv_path:str='../.env',
                path_to_weights:str=None,
                mlflow_model_alias:str="cycle1",
                mlflow_model_name:str="detector",
                mlflow_model_version:str=None,
                confidence_threshold:float=0.1):
        
        ## Load env variables
        load_dotenv(dotenv_path=dotenv_path)
        LABEL_STUDIO_URL = os.getenv('LABEL_STUDIO_URL')
        API_KEY = os.getenv("LABELSTUDIO-API-KEY")      
        self.labelstudio_client = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

        ## Load model from path
        self.tilesize=640
        self.overlapratio=0.1
        self.sahi_prostprocess='NMS'
        self.path_to_weights = path_to_weights
        if self.path_to_weights is None:
            ## Load  from mlflow
            TRACKING_URI="http://localhost:5000"
            mlflow.set_tracking_uri(TRACKING_URI)
            client = mlflow.MlflowClient()
            name = mlflow_model_name
            alias = mlflow_model_alias
            version = client.get_model_version_by_alias(name=name,alias=alias).version
            self.modelversion = f'{name}:{version}'
            self.modelURI = f'models:/{name}/{version}'
            self.model = mlflow.pyfunc.load_model(self.modelURI)
        else:
            device = "cuda" if torch.cuda.is_available() else "cpu"
            print('Device:', device)
            self.model = Yolov8DetectionModel(
                                                        model=YOLO(path_to_weights,task='detect'),
                                                        confidence_threshold=confidence_threshold,
                                                        image_size=self.tilesize,
                                                        device=device,
                                                        )
            self.modelversion = Path(path_to_weights).stem
            
        # LS label config
        self.from_name = "label"
        self.to_name = "image"
        self.label_type = "rectanglelabels"
        if mlflow_model_version is not None:
            self.modelversion = mlflow_model_version

    def predict(self, image:bytearray):

        if self.path_to_weights is None:
            return self.model.predict(image)
        
        result = get_sliced_prediction(image,
                                        self.model,
                                        slice_height=self.tilesize,
                                        slice_width=self.tilesize,
                                        overlap_height_ratio=self.overlapratio,
                                        overlap_width_ratio=self.overlapratio,
                                        postprocess_type=self.sahi_prostprocess,
                                        )
        return result.to_coco_annotations()

    def format_prediction(self,pred:dict,img_height:int,img_width:int):
        # formatting the prediction to work with Label studio
        x, y, width, height = pred['bbox']
        label = pred['category_name']
        score = pred['score']
        if not isinstance(score,float):
            score = 0.0
        template = {
                    "from_name": self.from_name,
                    "to_name": self.to_name,
                    "type": self.label_type,
                    "original_width":img_width,
                    "original_height":img_height,
                    "image_rotation":0,
                    'value': {
                        self.label_type: [label,],
                        'x': x / img_width * 100,
                        'y': y / img_height * 100,
                        'width': width / img_width * 100,
                        'height': height / img_height * 100,
                        'rotation':0
                    },
                    'score': score
        }
        return template
    
    def upload_predictions(self,project_id:int):

        # Select project
        project = self.labelstudio_client.get_project(id=project_id)

        # Upload predictions for each task
        tasks = project.get_tasks()
        for task in tqdm(tasks,desc="Uploading predictions"):
            task_id = task['id']
            img_url = task['data']['image']
            img_path = get_local_path(img_url)
            img = Image.open(img_path)
            prediction = self.predict(img)
            img_width, img_height = img.size
            formatted_pred = [self.format_prediction(pred,
                                                    img_height=img_height,
                                                    img_width=img_width) for pred in prediction]
            conf_scores = [pred['score'] for pred in prediction]
            max_score = 0.0
            if len(conf_scores)>0:
                max_score = max(conf_scores)
            project.create_prediction(task_id=task_id,
                                score=max_score,
                                result=formatted_pred,
                                model_version=self.modelversion)
        
    def build_upload_json(self,project_id:int,path_img_dir:str=None,root:str=None,
                          pattern="*.JPG",
                          bulk_predictions:list[dict]=None,
                          save_json_path:str=None):

        directory_preds = list()
        project = self.labelstudio_client.get_project(id=project_id)

        # Upload predictions for each task
        # for image_path in Path(path_img_dir).glob(pattern):
        #     d=image_path.relative_to(Path(root)).as_posix()
        tasks = project.get_tasks()
        for task in tqdm(tasks,desc="Uploading predictions"):
            img_url = task['data']['image']
            image_path = Path(get_local_path(img_url))
            pred = { 
                        "data": {"image" : img_url},
                        "predictions":[],
                    }
            # get predictions
            if bulk_predictions is None:
                start = time()
                image = Image.open(image_path)
                predictions = self.predict(image)
                print(f'Prediction time:{time() - start:.3f} seconds.')
                # format predictions
                img_width, img_height = image.size
                formatted_pred = [self.format_prediction(pred,
                                                        img_height=img_height,
                                                        img_width=img_width) for pred in predictions]
            else:
                predictions = bulk_predictions[image_path.name]
                formatted_pred = [self.format_prediction(pred,
                                                        img_height=pred['height'],
                                                        img_width=pred['width']) for pred in predictions]
            conf_scores = [pred['score'] for pred in predictions]
            # store predictions
            if len(conf_scores)>0:
                pred['predictions'].append({'result':formatted_pred,
                                            'model_version':self.modelversion,
                                            'score':max(conf_scores),
                                            }
                                            )
            else:
                pred['predictions'].append({'result':formatted_pred,
                                            'model_version':self.modelversion,
                                            'score':0.0
                                            }
                                            )
            # update buffer
            directory_preds.append(pred)

        if save_json_path is not None:
            with open(Path(save_json_path),'w') as file:
                json.dump(directory_preds,file,indent=2)

        return directory_preds

In [ ]:
# provide correct alias, "pt", "onnx"
handler = Annotator(mlflow_model_alias='cycle1')
# directory_preds = handler.build_upload_json(project_id=37)
handler.upload_predictions(project_id=37)

In [ ]:
# path_img_dir=r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images"
# save_json_path = os.path.join(Path(path_img_dir).parent,
#                               f"{Path(path_img_dir).name}_preannotation_label-studio.json")
# if save_json_path is not None:
#             with open(Path(save_json_path),'w') as file:
#                 json.dump(directory_preds,file,indent=2)

In [ ]:
# len(directory_preds)

# Inference with Sahi

In [ ]:
# from sahi.models.yolov8 import Yolov8DetectionModel
# from ultralytics import YOLO
# from sahi.predict import get_sliced_prediction
# import torch
from PIL import Image
from time import time

In [ ]:
class Detector(object):

    def __init__(self,
                path_to_weights:str,
                confidence_threshold:float=0.3):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.detection_model = Yolov8DetectionModel(
                                                    # model_path=path_to_weights,
                                                    model=YOLO(path_to_weights,task='detect'),
                                                    confidence_threshold=confidence_threshold,
                                                    image_size=640,
                                                    device=device,
                                                    )
        self.tilesize=640
        self.overlapratio=0.1
        self.sahi_prostprocess='NMS'
        print('Device:', device)
        
    def predict(self, image:str):
        image = Image.open(image)
        result = get_sliced_prediction(image, 
                                        self.detection_model,
                                        slice_height=self.tilesize,
                                        slice_width=self.tilesize,
                                        overlap_height_ratio=self.overlapratio,
                                        overlap_width_ratio=self.overlapratio,
                                        postprocess_type=self.sahi_prostprocess,
                                        ) 

        return result.to_coco_annotations()

In [ ]:
image_path = r"D:\PhD\Data per camp\Dry season\Kapiri\Camp 6-8\Rep 1 - tiled\DJI_20231003081043_0016_1.JPG"

In [ ]:
times = []
for ext in ['.pt','.onnx']:
    path = r"..\base_models_weights\yolov8.kaza" + ext
    model = Detector(path_to_weights=path,confidence_threshold=0.3)
    start = time()
    model.predict(image=image_path)
    times.append((ext,time() - start))

In [ ]:
times

In [ ]:
# img_path = Path(r"C:\Users\fadel\OneDrive\Bureau\e-savior\SAVMAP_samples\00a033fefe644429a1e0fcffe88f8b39.JPG")
# directory = img_path.parent/'preprocessed'
# directory.mkdir(parents=False,exist_ok=True)

In [ ]:
# data = imread(str(img_path))
# tilesize_h = 1000
# tilesize_w = 1000
# height, width, channels = data.shape 
# count = 0
# for i,j in tqdm(product(list(range(0,height,tilesize_h)),list(range(0,width,tilesize_w)))):
#     tile = data[i:min(i+tilesize_h,height),j:min(j+tilesize_w,width),:]
#     count += 1
#     filename = img_path.name.split('.')[0] + f'#{i}#{j}' + img_path.suffix
#     savepath = directory/filename
#     imsave(savepath,tile)
#     #assert sum(tile.shape) == tilesize_h+tilesize_w+channels,f"{tile.shape}"


In [ ]:
# plt.imshow(tile)
# plt.show()

In [ ]:
# height,width


# YOLO data_config.yaml 

In [ ]:
import yaml
import json
from arguments import Arguments

In [ ]:
# load yaml
with open(r"D:\PhD\Data per camp\IdentificationDataset\data_config.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)
yolo_config

In [ ]:
# load label mapping
args = Arguments()
with open(r"D:\PhD\Data per camp\IdentificationDataset\label_mapping.json",'r') as file:
    label_map = json.load(file)
names = [p['name'] for p in label_map if p['name'] not in args.discard_labels ]
label_map = dict(zip(range(len(names)),names))
label_map

In [ ]:
yolo_config.update({'names':label_map,'nc':len(label_map)})
yolo_config

In [ ]:
with open(r"D:\PhD\Data per camp\IdentificationDataset\data_config.yaml",'w') as file:
    yaml.dump(yolo_config,file,default_flow_style=False, sort_keys=False)

# Dataset distribution

In [ ]:
import yaml
import pandas as pd
import os
from pathlib import Path

In [ ]:
# load yaml
with open(r"D:\PhD\Data per camp\Extra training data\WAID\data_config.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)
yolo_config

In [ ]:
label_map = yolo_config['names']

In [ ]:
split = 'train'

path_dataset = os.path.join(yolo_config['path'],yolo_config[split][0])
path_dataset = path_dataset.replace('images','labels')

path_dataset

In [ ]:
labels = list()

for txtfile in Path(path_dataset).glob("*.txt"):

    df = pd.read_csv(txtfile,sep=" ",names = ['class','x','y','w','h'] )
    df['class'] = df['class'].astype(int)    
    df['image'] = txtfile.stem
    labels.append(df)


In [ ]:
df = pd.concat(labels,axis=0)
df['class'] = df['class'].map(label_map)

In [ ]:
images_per_class = dict()
for cls in df['class'].unique():
    num_imge = df.loc[df['class'] == cls,'image'].unique().shape[0]
    images_per_class[cls] = num_imge

In [ ]:
print("Split:", split)
print(images_per_class)

In [ ]:
print('Split:',split)
print(df['class'].value_counts())

In [ ]:
df['class'].value_counts().plot(kind='bar',figsize=(10,5),logy=True,title=f"{split} label distribution")

# Validation metrics

In [ ]:
from ultralytics import YOLO
from pathlib import Path

In [ ]:
# Load a model
path = r"C:/Users/Machine Learning/Desktop/workspace-wildAI/datalabeling/runs/mlflow/382537255263464058/5cc559b1a98d487983b3defbabe95c5f/artifacts/weights/best.pt"
model = YOLO(path)

In [ ]:
# Customize validation settings
validation_results = model.val(data=r"D:\PhD\Data per camp\IdentificationDataset\data_config.yaml", imgsz=640, batch=64, conf=0.25, iou=0.5, device="0")

# Sahi tutorials

In [ ]:
from sahi.slicing import slice_coco
from sahi.utils.file import load_json
from skimage.io import imread,imsave
import matplotlib.pyplot as plt
import os
import pandas as pd
import math
import shutil
from pathlib import Path
from arguments import Arguments
from utils import save_df_as_yolo, sample_data, get_slices, convert_json_annotations_to_coco,COCO_DIR_PATH,JSON_DIR_PATH, save_tiles, ALL_CSV
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
coco_dict = load_json(r"..\exported_annotations\coco-format\result.json")
coco_dict

In [ ]:
def load_coco_dataset(input_dir=COCO_DIR_PATH):

    def get_upload_img_dir(coco_annotation:dict):
        directory = set([os.path.dirname(metadata['file_name']) for metadata in coco_annotation['images']])
        assert len(directory)==1,'There should be one upload directory per annotation project'
        return directory.pop() #list(directory)[0]

    upload_img_dirs,coco_paths = list(),list()
    for path in Path(input_dir).glob('*.json'):
        annot = load_json(path)
        upload_img_dirs.append(get_upload_img_dir(coco_annotation=annot))
        coco_paths.append(path)
    
    return dict(zip(upload_img_dirs,coco_paths))

load_coco_dataset()


In [ ]:
def build_yolo_dataset(args:Arguments,ls_json_dir:str=JSON_DIR_PATH,clear_out_dir:bool=False,load_coco_existing:bool=False,ls_coco_dir:str=COCO_DIR_PATH):

    #clear directories
    if clear_out_dir:
        for p in [args.dest_path_images,args.dest_path_labels,COCO_DIR_PATH]:
            shutil.rmtree(p)
            Path(p).mkdir(parents=True,exist_ok=True)
    if load_coco_existing:
        map_imgdir_cocopath = load_coco_dataset(ls_coco_dir)
    else:
        # convert  json to coco
        map_imgdir_cocopath = convert_json_annotations_to_coco(input_dir=ls_json_dir)

    # slice coco annotations and save tiles
    for img_dir,cocopath in map_imgdir_cocopath.items():
        # slice annotations
        coco_dict_slices = get_slices(coco_annotation_file_path=cocopath,
                            img_dir=img_dir,
                            slice_height=args.height,
                            slice_width=args.width,
                            overlap_height_ratio=args.overlap_ratio,
                            overlap_width_ratio=args.overlap_ratio,
                            min_area_ratio=args.min_visibility
                            )
        # sample tiles
        df_tiles = sample_data(coco_dict_slices=coco_dict_slices,
                                empty_ratio=args.empty_ratio,
                                out_csv_path=None,
                                img_dir=img_dir,
                                labels_to_discard=args.discard_labels
                                )
        return df_tiles
        # save tiles
        # save_tiles(df_tiles=df_tiles,
        #            out_img_dir=args.dest_path_images,
        #            clear_out_img_dir=False)
        # # save labels in yolo format
        # save_df_as_yolo(df_annotation=df_tiles[~df_tiles['x'].isna()].copy(),
        #                 slice_height=args.height,
        #                 slice_width=args.width,
        #                 dest_path_labels=args.dest_path_labels)

        

In [ ]:
args = Arguments()
args.empty_ratio = 1
df_tiles = build_yolo_dataset(args=args,clear_out_dir=False,load_coco_existing=True)

In [ ]:
len(df_tiles) - 1537*2

In [ ]:
df_tiles.head(2)

In [ ]:
df_tiles['images'].iloc[:10].to_list()

In [ ]:
duplicated = df_tiles.duplicated(['x0','x1','y0','y1','images'])
df_tiles[~duplicated]

In [ ]:
df_tiles.duplicated(['images']).sum()

In [ ]:
num_annot = list()
for image_name,df in df_tiles.groupby('images'):
    # print(len(df),image_name)
    num_annot.append(len(df))
    # print(image_name)
    

In [ ]:
len(num_annot),max(num_annot),min(num_annot)

In [ ]:
2205+869

In [ ]:
sum(num_annot)

In [ ]:
plt.hist(num_annot,bins=13)
plt.show()

In [ ]:
df